###IMPORT PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

###IMPORT DATA

* Read Dataset

In [ ]:
#baca/buka file .csv
data = pd.read_csv("air_bnb.csv")

###DATA CLEANSING

* Hapus Data Yang Null/NaN lalu Tampilkan

In [ ]:
#lihat isi dataset
#hapus data yg NaN
data = data.dropna()
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ian,Mitte,Brunnenstr. Süd,52.534537,13.402557,Entire home/apt,60,4,118,2018-10-28,3.76,4,141
1,2695,Prenzlauer Berg close to Mauerpark,2986,Michael,Pankow,Prenzlauer Berg Nordwest,52.548513,13.404553,Private room,17,2,6,2018-10-01,1.42,1,0
2,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.534996,13.417579,Entire home/apt,90,62,143,2017-03-20,1.25,1,220
3,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.498855,13.349065,Private room,26,5,25,2018-08-16,0.39,1,297
4,7071,BrightRoom with sunny greenview!,17391,Bright,Pankow,Helmholtzplatz,52.543157,13.415091,Private room,42,2,197,2018-11-04,1.75,1,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22449,29772631,Cosy and lovely room near Ostkreuz,20727291,Anna,Lichtenberg,Neu Lichtenberg,52.502952,13.490870,Private room,25,2,1,2018-11-04,1.00,1,32
22463,29784461,House on the water HT10 - Heart of East Berlin,22437698,Sebastian,Treptow - Köpenick,Oberschöneweide,52.457982,13.526945,Entire home/apt,99,1,5,2018-11-06,5.00,2,339
22475,29792124,Welcome to Light Apartment,224133729,Inna,Friedrichshain-Kreuzberg,Frankfurter Allee Süd FK,52.507884,13.467071,Private room,25,1,1,2018-11-05,1.00,1,8
22508,29817383,Prenzlauerberg,224295751,Salem,Pankow,Prenzlauer Berg Nordwest,52.552250,13.410904,Entire home/apt,70,1,1,2018-11-07,1.00,1,350


#EKSPERIMEN 1

###DATA EXPLORATION

* Encode value room type dari string menjadi integer

In [ ]:
# fungsi untuk mengubah room_type menjadi angka
def to_numerical(data):
    kamus = {k: i for i, k in enumerate(data.unique())}
    return [kamus[d] for d in data]

* Ambil fitur atau beberapa kolom yang diperlukan

In [ ]:
# klasifikasi room_type
# ambil fitur yang diperlukan
df = pd.DataFrame({
    "price": data.price,
    "minimum_nights": data.minimum_nights,
    "number_of_reviews": data.number_of_reviews,
    "calculated_host_listings_count": data.calculated_host_listings_count,
    "availability_365": data.availability_365,
    "latitude": data.latitude,
    "longitude": data.longitude,
    "room_type": to_numerical(data.room_type)
})

* Tampilkan kolom yang dipilih

In [ ]:
df

,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,latitude,longitude,room_type
0,60,4,118,4,141,52.534537,13.402557,0
1,17,2,6,1,0,52.548513,13.404553,1
2,90,62,143,1,220,52.534996,13.417579,0
3,26,5,25,1,297,52.498855,13.349065,1
4,42,2,197,1,26,52.543157,13.415091,1
...,...,...,...,...,...,...,...,...
22449,25,2,1,1,32,52.502952,13.490870,1
22463,99,1,5,2,339,52.457982,13.526945,0
22475,25,1,1,1,8,52.507884,13.467071,1
22508,70,1,1,1,350,52.552250,13.410904,0


###Classification

#####DATA SPLITTING

* Memisahkan Antara Data Latih dan(X) dan Label(Y), dengan data latih atribut "room_type"

In [ ]:
# pisahkan data latih dan label
X = np.array(df.drop("room_type", 1))
Y = df.price.values

* Setelah dipisahkan menjadi data latih(X) dan data uji(Y), validasi data uji menjadi 1000

In [ ]:
# pisahkan data menjadi data latih dan data uji (1000 data uji)
X_test = X[:1000]
Y_test = Y[:1000]
X = X[1000:]
Y = Y[1000:]

* Untuk mencari error model, disini mmenggunakan MSE(Mean Square Error). metode ini mengevaluasi metode peramalan. Masing-masing kesalahan atau sisa dikuadratkan. Kemudian dijumlahkan dan ditambahkan dengan jumlah observasi. Pendekatan ini mengatur kesalahan peramalan yang besar karena kesalahan-kesalahan itu dikuadratkan.

In [ ]:
#evaluasi menggunakan mse untuk mencari error
def mse(prediksi, target):
    return np.square(np.subtract(prediksi, target)).mean()

###NAIVE BAYES

* Algoritma ini mengklasifikasikan data dengan cara memprediksi data yang tidak diketahui
* lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan gnb.fit(X, Y)

In [ ]:
#buat model menggunakan algoritma navie bayes
gnb = GaussianNB()
gnb.fit(X, Y)

GaussianNB(priors=None, var_smoothing=1e-09)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE 

In [ ]:
#hitung akurasi dan nilai error
prediksi = gnb.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.984
Mean Square Error: 0.398


###RANDOM FOREST

* Random Forest dilakukan melalui penggabungan pohon (tree) dengan melakukan training pada sampel data yang dimiliki. Penentuan klasifikasi dengan random forest diambil berdasarkan hasil voting dari tree yang terbentuk.
* lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan clf.fit(X, Y)

In [ ]:
#menggunakan algoritma random forest
clf = RandomForestClassifier(n_estimators=10)
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE



In [ ]:
#hitung akurasinya
prediksi = clf.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.553
Mean Square Error: 1375.231


### DECISION TREE

* Decision Tree dimulai dari root node hingga leaf node yang dilakukan secara rekursif. Di mana setiap percabangan menyatakan suatu kondisi yang harus dipenuhi dan pada setiap ujung pohon menyatakan kelas dari suatu data.
*lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan dtc.fit(X, Y)

In [ ]:
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(X, Y)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE

In [ ]:
#hitung akurasinya
prediksi = dtc.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.986
Mean Square Error: 3.168


#EKSPERIMEN 2

###DATA EXPLORATION

* Encode value room type dari string menjadi integer

In [ ]:
# fungsi untuk mengubah room_type menjadi angka
def to_numerical(data):
    kamus = {k: i for i, k in enumerate(data.unique())}
    return [kamus[d] for d in data]

* Ambil fitur atau beberapa kolom yang diperlukan

In [ ]:
# klasifikasi room_type
# ambil fitur yang diperlukan
df = pd.DataFrame({
    "price": data.price,
    "name": to_numerical(data.room_type),
    "host_name": to_numerical(data.host_name),
    "calculated_host_listings_count": data.calculated_host_listings_count,
    "neighbourhood": to_numerical(data.neighbourhood),
    "room_type": to_numerical(data.room_type)
})

* Tampilkan kolom yang dipilih

In [ ]:
df

,price,name,host_name,calculated_host_listings_count,neighbourhood,room_type
0,60,0,0,4,0,0
1,17,1,1,1,1,1
2,90,0,2,1,2,0
3,26,1,3,1,3,1
4,42,1,4,1,4,1
...,...,...,...,...,...,...
22449,25,1,139,1,50,1
22463,99,0,31,2,116,0
22475,25,1,3725,1,10,1
22508,70,0,5237,1,1,0


###Classification

#####DATA SPLITTING

* Memisahkan Antara Data Latih dan(X) dan Label(Y), dengan data latih atribut "name"

In [ ]:
# pisahkan data latih dan label
X = np.array(df.drop("name", 1))
Y = df.price.values

* Setelah dipisahkan menjadi data latih(X) dan data uji(Y), validasi data uji menjadi 1000

In [ ]:
# pisahkan data menjadi data latih dan data uji (1000 data uji)
X_test = X[:1000]
Y_test = Y[:1000]
X = X[1000:]
Y = Y[1000:]

* Untuk mencari error model, disini mmenggunakan MSE(Mean Square Error). metode ini mengevaluasi metode peramalan. Masing-masing kesalahan atau sisa dikuadratkan. Kemudian dijumlahkan dan ditambahkan dengan jumlah observasi. Pendekatan ini mengatur kesalahan peramalan yang besar karena kesalahan-kesalahan itu dikuadratkan.

In [ ]:
#evaluasi menggunakan mse untuk mencari error
def mse(prediksi, target):
    return np.square(np.subtract(prediksi, target)).mean()

###NAIVE BAYES

* Algoritma ini mengklasifikasikan data dengan cara memprediksi data yang tidak diketahui
* lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan gnb.fit(X, Y)

In [ ]:
#buat model menggunakan algoritma navie bayes
gnb = GaussianNB()
gnb.fit(X, Y)

GaussianNB(priors=None, var_smoothing=1e-09)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE 

In [ ]:
#hitung akurasi dan nilai error
prediksi = gnb.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.986
Mean Square Error: 5.287


###RANDOM FOREST

* Random Forest dilakukan melalui penggabungan pohon (tree) dengan melakukan training pada sampel data yang dimiliki. Penentuan klasifikasi dengan random forest diambil berdasarkan hasil voting dari tree yang terbentuk.
* lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan clf.fit(X, Y)

In [ ]:
#menggunakan algoritma random forest
clf = RandomForestClassifier(n_estimators=10)
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE



In [ ]:
#hitung akurasinya
prediksi = clf.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.803
Mean Square Error: 5303.312


### DECISION TREE

* Decision Tree dimulai dari root node hingga leaf node yang dilakukan secara rekursif. Di mana setiap percabangan menyatakan suatu kondisi yang harus dipenuhi dan pada setiap ujung pohon menyatakan kelas dari suatu data.
*lalu latih classifiernya terhadap data X agar menghasilkan output Y dengan dtc.fit(X, Y)

In [ ]:
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(X, Y)

* Hitung Akurasi dengan  mencari hasil output data uji, lalu hasilnya dibandingkan dengan output sebenarnya yaitu label uji dan MSE

In [ ]:
#hitung akurasinya
prediksi = dtc.predict(X_test)
score = 0
for p, t in zip(prediksi, Y_test):
    if p == t:
        score += 1
print('Akurasi:', score/1000)
print('Mean Square Error:', mse(prediksi, Y_test))

Akurasi: 0.987
Mean Square Error: 1563.248


##KESIMPULAN
* Hasil Klasifikasi ditentukan oleh fitur fitur yang sudah kita tentukan atau kita pilih, atribut dataset yang kita pilih menjadi data latih dan data uji, juga mempengaruhi hasil 
* Atribut dengan value unik yang sedikit juga mempengaruhi proses klasifikasi dan akurasi
* Metode atau classifier yang berbeda beda, menghasilkan nilai akurasi yang berbeda beda
* Dari tiga metode atau classifier yang digunakan, yaitu Naive Bayes, Random Forest, dan Decision Tree, setelah dilakukan dua kali ekperiman, dari nilai akurasinya dibuktikan bahwa algoritma yang baik untuk proses klasifikasi adalah algoritma Decision Tree